# Ensemble Methods - Padmaja 19BAI1107

In [1]:
import pandas as pd
import numpy as np
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report,confusion_matrix,homogeneity_score,completeness_score,v_measure_score

### Importing Data

In [6]:
dftrain=pd.read_excel('Multiclass-dataset.xlsx', sheet_name='trainingset',engine='openpyxl')
dftest=pd.read_excel('Multiclass-dataset.xlsx', sheet_name='TestingSet', engine='openpyxl')

In [7]:
dftrain.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P12,P13,P14,P15,P16,P17,P18,P19,P20,Target Label
0,3.30,7.44,1.52,3.27,0.07,2.14,0.75,0.66,0.0,54.8,...,50.7,6.55,4.09,4.26,0.01,0.00,24.7,2.7,1.6,V1
1,3.43,7.63,1.63,3.27,0.05,2.01,0.74,0.65,0.0,51.8,...,47.9,8.35,5.08,5.01,0.01,0.00,23.3,2.3,1.8,V1
2,3.41,7.32,1.52,3.18,0.07,2.09,0.80,0.70,0.0,54.0,...,54.4,9.27,6.85,7.14,0.19,0.06,25.0,2.5,-0.9,V1
3,3.78,7.85,1.69,3.35,0.03,1.98,0.77,0.67,0.0,57.7,...,48.9,10.26,5.96,5.47,0.05,0.01,24.1,5.6,2.1,V1
4,3.90,7.99,1.61,3.43,0.02,2.14,0.77,0.71,0.0,59.1,...,54.1,8.19,5.81,4.72,0.64,0.16,26.8,2.5,2.1,V1


In [8]:
dftest.head()

,P1,P2,P3,P4,P5,P6,P7,P8,P9,P10,...,P12,P13,P14,P15,P16,P17,P18,P19,P20,Target Label
0,2.85,6.98,1.47,3.03,0.04,2.06,0.74,0.64,0.0,57.1,...,52.7,8.94,6.17,6.43,0.19,0.07,25.3,3.7,1.0,V1
1,3.90,7.92,1.72,3.40,0.05,1.97,0.78,0.66,0.0,63.7,...,54.8,11.04,8.05,7.70,0.71,0.18,27.2,5.5,2.1,V1
2,3.31,7.61,1.44,3.37,0.04,2.34,0.72,0.68,0.0,61.0,...,50.7,7.63,4.85,4.76,0.07,0.02,25.7,5.3,3.1,V1
3,3.71,7.71,1.61,3.27,0.05,2.03,0.79,0.71,0.0,61.7,...,55.6,8.74,6.43,6.56,0.76,0.20,27.4,3.7,1.8,V1
4,3.26,7.42,1.44,3.27,0.05,2.27,0.74,0.69,0.0,61.6,...,53.6,9.89,6.97,6.83,0.34,0.11,26.7,4.6,2.2,V1


In [9]:
dict_diff={
    'V1':1,
    'V2':2,
    'V3':3,
    'V4':4,
    'V5':5,
    'V6':6,
    'V7':7,
    'V8':8,
    'V9':9,
    'V10':10
    }

### Changing the Target Label 

In [10]:
dftest['Target Label']=dftest['Target Label'].apply(lambda x:dict_diff[x])
dftrain['Target Label']=dftrain['Target Label'].apply(lambda x:dict_diff[x])

In [11]:
features=dftrain.loc[:,dftrain.columns!='Target Label'].values[:,1:]
labels=dftrain.loc[:,'Target Label'].values
labels

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
        6,  6,  6,  6,  6

In [12]:
features1=dftest.loc[:,dftest.columns!='Target Label'].values[:,1:]
labels1=dftest.loc[:,'Target Label'].values
labels1

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,
        6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,
        7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10],
      dtype=int64)

### Data Preprocessing using MinMax Scaler

In [13]:
scaler=MinMaxScaler((-1,1))
x_train=scaler.fit_transform(features)
y_train=labels
x_train.shape

(400, 19)

In [14]:
type(x_train)

numpy.ndarray

In [15]:
type(y_train)

numpy.ndarray

In [16]:
scaler=MinMaxScaler((-1,1))
x_test=scaler.fit_transform(features1)
y_test=labels1
x_test.shape

(100, 19)

# BOOSTING

## 1. XGBoost

### Model 1

In [17]:
model1 = XGBClassifier()
model1.fit(x_train, y_train)

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:49:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
y_pred1 = model1.predict(x_test)

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


In [19]:
accuracy_score(y_test, y_pred1)

0.74

### Model 2

In [20]:
param_grid = {'booster':['gbtree','gblinear','dart'],'max_depth':[1,2,3,4,5,6],'sampling_method':['uniform','gradient_based']}

In [21]:
grid = GridSearchCV(XGBClassifier(),param_grid,refit=True,verbose=3)

In [22]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[20:50:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 1/5] END booster=gbtree, max_depth=1, sampling_method=uniform; total time=   0.5s
[20:50:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 2/5] END booster=gbtree, max_depth=1, sampling_method=uniform; total time=   0.5s
[20:50:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV 4/5] END booster=dart, max_depth=6, sampling_method=gradient_based; total time=  11.1s
[21:02:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV 5/5] END booster=dart, max_depth=6, sampling_method=gradient_based; total time=  11.7s
[21:02:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Dell\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, validate_parameter

In [23]:
grid.best_params_

{'booster': 'gbtree', 'max_depth': 6, 'sampling_method': 'uniform'}

In [24]:
model2=grid.best_estimator_

In [25]:
y_pred2= model2.predict(x_test)

C:\Users\Dell\anaconda3\lib\site-packages\xgboost\data.py:115: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "memory consumption")


In [26]:
accuracy_score(y_test, y_pred2)

0.74

## 2. Adaboost

### Model 1

In [27]:
abc = AdaBoostClassifier(n_estimators=50,learning_rate=1)
model3 = abc.fit(x_train, y_train)
y_pred3 = model3.predict(x_test)

In [28]:
accuracy_score(y_test, y_pred3)

0.39

### Model 2

In [29]:
param_grid = {'n_estimators':[10,20,30,40,50],'learning_rate':[1,2,3,4,5],'algorithm':['SAMME','SAMME.R']}

In [30]:
grid = GridSearchCV(AdaBoostClassifier(),param_grid,refit=True,verbose=3)

In [31]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END algorithm=SAMME, learning_rate=1, n_estimators=10; total time=   0.0s
[CV 2/5] END algorithm=SAMME, learning_rate=1, n_estimators=10; total time=   0.0s
[CV 3/5] END algorithm=SAMME, learning_rate=1, n_estimators=10; total time=   0.0s
[CV 4/5] END algorithm=SAMME, learning_rate=1, n_estimators=10; total time=   0.0s
[CV 5/5] END algorithm=SAMME, learning_rate=1, n_estimators=10; total time=   0.0s
[CV 1/5] END algorithm=SAMME, learning_rate=1, n_estimators=20; total time=   0.0s
[CV 2/5] END algorithm=SAMME, learning_rate=1, n_estimators=20; total time=   0.0s
[CV 3/5] END algorithm=SAMME, learning_rate=1, n_estimators=20; total time=   0.0s
[CV 4/5] END algorithm=SAMME, learning_rate=1, n_estimators=20; total time=   0.0s
[CV 5/5] END algorithm=SAMME, learning_rate=1, n_estimators=20; total time=   0.0s
[CV 1/5] END algorithm=SAMME, learning_rate=1, n_estimators=30; total time=   0.0s
[CV 2/5] END algorithm=SA

[CV 4/5] END algorithm=SAMME, learning_rate=4, n_estimators=50; total time=   0.1s
[CV 5/5] END algorithm=SAMME, learning_rate=4, n_estimators=50; total time=   0.1s
[CV 1/5] END algorithm=SAMME, learning_rate=5, n_estimators=10; total time=   0.0s
[CV 2/5] END algorithm=SAMME, learning_rate=5, n_estimators=10; total time=   0.0s
[CV 3/5] END algorithm=SAMME, learning_rate=5, n_estimators=10; total time=   0.0s
[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=10; total time=   0.0s
[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=10; total time=   0.0s
[CV 1/5] END algorithm=SAMME, learning_rate=5, n_estimators=20; total time=   0.0s
[CV 2/5] END algorithm=SAMME, learning_rate=5, n_estimators=20; total time=   0.0s
[CV 3/5] END algorithm=SAMME, learning_rate=5, n_estimators=20; total time=   0.0s
[CV 4/5] END algorithm=SAMME, learning_rate=5, n_estimators=20; total time=   0.0s
[CV 5/5] END algorithm=SAMME, learning_rate=5, n_estimators=20; total time=   0.0s
[CV 

[CV 2/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; total time=   0.1s
[CV 3/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; total time=   0.2s
[CV 4/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; total time=   0.2s
[CV 5/5] END algorithm=SAMME.R, learning_rate=3, n_estimators=50; total time=   0.1s
[CV 1/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=10; total time=   0.0s
[CV 2/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=10; total time=   0.0s
[CV 3/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=10; total time=   0.0s
[CV 4/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=10; total time=   0.0s
[CV 5/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=10; total time=   0.0s
[CV 1/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=20; total time=   0.0s
[CV 2/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=20; total time=   0.0s
[CV 3/5] END algorithm=SAMME.R, learning_rate=4, n_estimators=20;

GridSearchCV(estimator=AdaBoostClassifier(),
             param_grid={'algorithm': ['SAMME', 'SAMME.R'],
                         'learning_rate': [1, 2, 3, 4, 5],
                         'n_estimators': [10, 20, 30, 40, 50]},
             verbose=3)

In [32]:
grid.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 2, 'n_estimators': 50}

In [33]:
model4=grid.best_estimator_

In [34]:
y_pred4= model4.predict(x_test)

In [35]:
accuracy_score(y_test, y_pred4)

0.66

## 3. Gradient Descent

### Model 1

In [36]:
model5 = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
model5.fit(x_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=1)

In [38]:
y_pred5= model5.predict(x_test)

In [39]:
accuracy_score(y_test, y_pred5)

0.65

### Model 2

In [40]:
param_grid={'loss':[ 'deviance','exponential'],'learning_rate':[1,2,3,4,5],'criterion':['friedman_mse', 'mse', 'mae'],'max_features':['auto', 'sqrt', 'log2']}

In [41]:
grid = GridSearchCV(GradientBoostingClassifier(),param_grid,refit=True,verbose=3)

In [42]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=auto; total time=   2.6s
[CV 2/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=auto; total time=   2.5s
[CV 3/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=auto; total time=   2.6s
[CV 4/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=auto; total time=   2.6s
[CV 5/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=auto; total time=   2.5s
[CV 1/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 2/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=sqrt; total time=   1.5s
[CV 3/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 4/5] END criterion=friedman_mse, learning_rate=1, loss=deviance, max_features=

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 249, in _check_params
    self.loss_ = loss_class(self.n_classes_)
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py", line 776, in __init__
    .format(self.__class__.__name__, n_classes))
ValueError: ExponentialLoss requires 2 classes; got 10 class(es)

  FitFailedWarning)
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validat

[CV 1/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=auto; total time=   2.5s
[CV 2/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=auto; total time=   2.4s
[CV 3/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=auto; total time=   2.7s
[CV 4/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=auto; total time=   2.6s
[CV 5/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=auto; total time=   2.4s
[CV 1/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 2/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=sqrt; total time=   1.5s
[CV 3/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 4/5] END criterion=friedman_mse, learning_rate=2, loss=deviance, max_features=sqrt; total time=   1.5s
[CV 5/5] END criterion=friedman_mse, 

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 249, in _check_params
    self.loss_ = loss_class(self.n_classes_)
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb_losses.py", line 776, in __init__
    .format(self.__class__.__name__, n_classes))
ValueError: ExponentialLoss requires 2 classes; got 10 class(es)

  FitFailedWarning)
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validat

[CV 1/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=auto; total time=   2.3s
[CV 2/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=auto; total time=   2.3s
[CV 3/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=auto; total time=   2.6s
[CV 4/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=auto; total time=   2.6s
[CV 5/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=auto; total time=   2.6s
[CV 1/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=sqrt; total time=   1.9s
[CV 2/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 3/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=sqrt; total time=   1.5s
[CV 4/5] END criterion=friedman_mse, learning_rate=3, loss=deviance, max_features=sqrt; total time=   1.6s
[CV 5/5] END criterion=friedman_mse, 

...

[CV 4/5] END criterion=mae, learning_rate=5, loss=deviance, max_features=log2; total time=   4.7s


C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1122: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  "Boosting.", FutureWarning)


[CV 5/5] END criterion=mae, learning_rate=5, loss=deviance, max_features=log2; total time=   4.9s
[CV 1/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=auto; total time=   0.0s
[CV 2/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=auto; total time=   0.0s
[CV 3/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=auto; total time=   0.0s
[CV 4/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=auto; total time=   0.0s
[CV 5/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=auto; total time=   0.0s
[CV 1/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=sqrt; total time=   0.0s
[CV 2/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=sqrt; total time=   0.0s
[CV 3/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=sqrt; total time=   0.0s
[CV 4/5] END criterion=mae, learning_rate=5, loss=exponential, max_features=sqrt; total time= 

C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1122: FutureWarning: criterion='mae' was deprecated in version 0.24 and will be removed in version 1.1 (renaming of 0.26). Use criterion='friedman_mse' or 'mse' instead, as trees should use a least-square criterion in Gradient Boosting.
  "Boosting.", FutureWarning)
C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 448, in fit
    self._check_params()
  File "C:\Users\Dell\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py", line 249, in _check_params
    self.loss_ = loss_clas

GridSearchCV(estimator=GradientBoostingClassifier(),
             param_grid={'criterion': ['friedman_mse', 'mse', 'mae'],
                         'learning_rate': [1, 2, 3, 4, 5],
                         'loss': ['deviance', 'exponential'],
                         'max_features': ['auto', 'sqrt', 'log2']},
             verbose=3)

In [43]:
grid.best_params_

{'criterion': 'mse',
 'learning_rate': 1,
 'loss': 'deviance',
 'max_features': 'auto'}

In [44]:
model6=grid.best_estimator_

In [45]:
y_pred6=model6.predict(x_test)

In [46]:
accuracy_score(y_test, y_pred6)

0.61

# Bagging Classification

### Model 1

In [47]:
seed = 10
kfold = model_selection.KFold(n_splits = 3,shuffle=True,
                       random_state = seed)

In [48]:
base_cls = DecisionTreeClassifier()
num_trees = 500
model7 = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = num_trees,
                          random_state = seed)

In [49]:
model_selection.cross_val_score(model7, x_train, y_train, cv = kfold).mean()

0.7475404930236037

In [50]:
model7.fit(x_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=500,
                  random_state=10)

In [51]:
y_pred7=model7.predict(x_test)

In [52]:
accuracy_score(y_test, y_pred7)

0.77

### Model 2

In [53]:
seed = 9
kfold = model_selection.KFold(n_splits = 3,shuffle=True,
                       random_state = seed)

In [54]:
base_cls = SVC()
num_trees = 500
model8 = BaggingClassifier(base_estimator = base_cls,
                          n_estimators = num_trees,
                          random_state = seed)

In [55]:
model_selection.cross_val_score(model8, x_train, y_train, cv = kfold).mean()

0.6926083866382373

In [56]:
model8.fit(x_train,y_train)

BaggingClassifier(base_estimator=SVC(), n_estimators=500, random_state=9)

In [57]:
y_pred8=model8.predict(x_test)

In [58]:
accuracy_score(y_test, y_pred8)

0.69

# Random Forests

### Model 1

In [59]:
model9 = RandomForestClassifier(max_depth=2, random_state=0)

In [60]:
model9.fit(x_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [61]:
y_pred9=model9.predict(x_test)

In [62]:
accuracy_score(y_test, y_pred9)

0.68

### Model 2

In [63]:
param_grid={'n_estimators':[100,200,300,400],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [64]:
grid = GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3)

In [65]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=2, max_features=a

[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=100; total time=   0.1s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=200; total time=   0.4s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.5s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.6s
[CV 4/5] END criteri

[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.4s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=200; total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.6s
[CV 3/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.6s
[CV 4/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.5s
[CV 5/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=300; total time=   0.6s
[CV 1/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.8s
[CV 2/5] END criterion=gini, max_depth=4, max_features=log2, n_estimators=400; total time=   0.8s
[CV 3/5] END criteri

[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.6s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.6s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.6s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=300; total time=   0.7s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.8s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.8s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=auto, n_estimators=400; total time=   0.9s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=sqrt, n_estimators=100; total tim

[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.7s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=300; total time=   0.7s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.0s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   0.9s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.0s
[CV 4/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.0s
[CV 5/5] END criterion=entropy, max_depth=3, max_features=sqrt, n_estimators=400; total time=   1.0s
[CV 1/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=3, max_features=log2, n_estimators=100; total tim

[CV 1/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=log2, n_estimators=400; total time=   1.1s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.2s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.2s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.2s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total time=   0.2s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=100; total tim

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [100, 200, 300, 400]},
             verbose=3)

In [66]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'auto',
 'n_estimators': 200}

In [67]:
model10=grid.best_estimator_

In [68]:
y_pred10= model10.predict(x_test)

In [69]:
accuracy_score(y_test, y_pred10)

0.77

# Extra Trees Classification

### Model 1

In [70]:
extra_tree_forest = ExtraTreesClassifier(n_estimators = 5,
                                        criterion ='entropy', max_features = 2)
extra_tree_forest.fit(x_train, y_train)
feature_importance = extra_tree_forest.feature_importances_
feature_importance_normalized = np.std([tree.feature_importances_ for tree in 
                                        extra_tree_forest.estimators_],
                                        axis = 0)

In [118]:
model11=extra_tree_forest

In [119]:
y_pred11=model11.predict(x_test)

In [120]:
accuracy_score(y_test,y_pred11)


0.64

### Model 2

In [73]:
param_grid={'n_estimators':[5,6,7,8,9],'criterion':['gini', 'entropy'],'max_depth':[2,3,4,5],'max_features':['auto','sqrt','log2']}

In [74]:
grid = GridSearchCV(ExtraTreesClassifier(),param_grid,refit=True,verbose=3)

In [75]:
grid.fit(x_train,y_train)

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=2, max_features=auto, n_estimators

[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=3, max_features=auto, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=3, max_features=sqrt, n_estimators=5; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=3

[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=4, max_features=sqrt, n_estimators=9; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=4

[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=5, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=gini, max_depth=5

[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=2, max_features=log2, n_estimators=9; total time=   0.0s
[CV 4/5] E

[CV 5/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=5; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=6; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=6; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=6; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=6; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=6; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=7; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=7; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=7; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=4, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] E

[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=7; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 4/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 5/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=8; total time=   0.0s
[CV 1/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=9; total time=   0.0s
[CV 2/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=9; total time=   0.0s
[CV 3/5] END criterion=entropy, max_depth=5, max_features=auto, n_estimators=9; total time=   0.0s
[CV 4/5] E

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [5, 6, 7, 8, 9]},
             verbose=3)

In [76]:
grid.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_features': 'log2',
 'n_estimators': 9}

In [115]:
model12=grid.best_estimator_

In [116]:
y_pred12= model12.predict(x_test)

In [117]:
accuracy_score(y_test, y_pred12)

0.7

# Results and Inference

In [122]:
print('Model\t\t\t\t\t\t\t\t\t\t\tAccuracy\n')
print('XGB Classifier model 1\t\t\t\t\t\t\t\t\t',accuracy_score(y_test,y_pred1))
print('XGB Classifier model 2\t\t\t\t\t\t\t\t\t',accuracy_score(y_test,y_pred2))
print(model3,'\t\t\t\t\t\t\t',accuracy_score(y_test,y_pred3))
print(model4,'\t\t\t\t\t\t\t',accuracy_score(y_test,y_pred4))
print(model5,'\t\t\t\t',accuracy_score(y_test,y_pred5))
print(model6,'\t\t\t\t\t',accuracy_score(y_test,y_pred6))
print(model7,'\t\t\t\t\t\t\t',accuracy_score(y_test,y_pred7))
print(model8,'\t\t',accuracy_score(y_test,y_pred8))
print(model9,'\t\t\t\t\t',accuracy_score(y_test,y_pred9))
print(model10,'\t\t\t\t\t',accuracy_score(y_test,y_pred10))
print(model11,'\t\t',accuracy_score(y_test,y_pred11))
print(model12,'\t\t\t',accuracy_score(y_test,y_pred12))


Model											Accuracy

XGB Classifier model 1									 0.74
XGB Classifier model 2									 0.74
AdaBoostClassifier(learning_rate=1) 							 0.39
AdaBoostClassifier(learning_rate=2) 							 0.66
GradientBoostingClassifier(learning_rate=1.0, max_depth=1) 				 0.65
GradientBoostingClassifier(criterion='mse', learning_rate=1,
                           max_features='auto') 					 0.61
BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=500,
                  random_state=10) 							 0.77
BaggingClassifier(base_estimator=SVC(), n_estimators=500, random_state=9) 		 0.69
RandomForestClassifier(max_depth=2, random_state=0) 					 0.68
RandomForestClassifier(max_depth=5, n_estimators=200) 					 0.77
ExtraTreesClassifier(criterion='entropy', max_features=2, n_estimators=5) 		 0.64
ExtraTreesClassifier(max_depth=5, max_features='log2', n_estimators=9) 			 0.7


### Inference : Thus for the given dataset BaggingClassifier - Model 1 and RandomForestClassifier - Model 2 give highest accuracies of 77%.